<a href="https://colab.research.google.com/github/JstnClmnt/NLP-Text-Summarization/blob/practice/NLP_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import re
np.random.seed(1)

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import pandas as pd
df=pd.read_csv("/content/drive/Team Drives/AI Lords/NLP-Text-Summarization/data/news_summary_more.csv",encoding="latin-1")
df.head()


,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [0]:
df['text'] = df['text'].str.replace("[^a-zA-Z#]"," ")
df['headlines'] = df['headlines'].str.replace("[^a-zA-Z#]"," ")
df=df.dropna()

In [0]:
df=df.sample(n=1500)

In [6]:
from keras.preprocessing.text import Tokenizer
VOCAB_SIZE = 10000
tokenizer = Tokenizer(oov_token=1,num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(df["text"])
article_sequences = tokenizer.texts_to_sequences(df["text"])
art_word_index = tokenizer.word_index
len(art_word_index)


Using TensorFlow backend.


11698

In [7]:
art_word_index_1500 = {}
counter = 0
for word in art_word_index.keys():
    if art_word_index[word] == 0:
        print("found 0!")
        break
    if art_word_index[word] > VOCAB_SIZE:
        continue
    else:
        art_word_index_1500[word] = art_word_index[word]
        counter += 1
print(counter)

10000


In [8]:
tokenizer.fit_on_texts(df["headlines"])
summary_sequences = tokenizer.texts_to_sequences(df["headlines"])
sum_word_index = tokenizer.word_index
len(sum_word_index)

12098

In [9]:
sum_word_index_1500 = {}
counter = 0
for word in sum_word_index.keys():
    if sum_word_index[word] == 0:
        print("found 0!")
        break
    if sum_word_index[word] > VOCAB_SIZE:
        continue
    else:
        sum_word_index_1500[word] = sum_word_index[word]
        counter += 1
print(counter)

10000


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(article_sequences, summary_sequences, test_size=0.20, random_state=42)

print(X_train[0])

[7677, 10, 21, 14, 4900, 220, 426, 726, 1890, 839, 104, 3, 1661, 62, 1510, 1172, 5, 7678, 2, 1076, 134, 68, 11, 413, 25, 1345, 109, 41, 4901, 366, 4902, 5, 2, 7679, 791, 2, 328, 248, 91, 8, 4, 2359, 1028, 4903, 6, 515, 7680, 8, 2865, 53, 4904, 1418, 1159, 7, 2746, 1011, 3, 2, 3653]


In [11]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2019-05-18 20:06:01--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-05-18 20:06:01--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1       27%[====>               ] 228.07M  6.97MB/s    eta 67s    ^C


In [12]:
!unzip "glove.6B.zip"

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
import numpy as np

embeddings_index = dict()
f = open('glove.6B.300d.txt',encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [14]:
EMB_DIM=300
num_words=VOCAB_SIZE+1
print("Number of Words:"+str(num_words))

MAX_LENGTH = len(max(X_train, key=len))
print("Max Length: "+str(MAX_LENGTH))  # 271
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
train_sentences_X = pad_sequences(X_train, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(X_test, maxlen=MAX_LENGTH, padding='post')
train_sentences_Y= pad_sequences(y_train, maxlen=MAX_LENGTH, padding='post')
test_sentences_Y= pad_sequences(y_test, maxlen=MAX_LENGTH, padding='post')
#y_train=to_categorical(y_train)
#y_test=to_categorical(y_test)
print(train_sentences_X[0])
print(train_sentences_Y[0])

Number of Words:10001
Max Length: 68
[7677   10   21   14 4900  220  426  726 1890  839  104    3 1661   62
 1510 1172    5 7678    2 1076  134   68   11  413   25 1345  109   41
 4901  366 4902    5    2 7679  791    2  328  248   91    8    4 2359
 1028 4903    6  515 7680    8 2865   53 4904 1418 1159    7 2746 1011
    3    2 3653    0    0    0    0    0    0    0    0    0]
[4282  490    9  706   91    8 1324 1306    4 5735    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


In [15]:
embedding_matrix_art=np.zeros((num_words,EMB_DIM))
numNoEmb=0
#print(word2index)
for word,i in art_word_index_1500.items():
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_art[i]=embedding_vector
    else:
      numNoEmb+=1
print(numNoEmb)

476


In [16]:
embedding_matrix_sum=np.zeros((num_words,EMB_DIM))
#print(word2index)
numNoEmb=0
for word,i in sum_word_index_1500.items():
    if i>num_words:
        continue
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_sum[i]=embedding_vector
    else:
      numNoEmb+=1
print(numNoEmb)

468


In [24]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Input,Dense, CuDNNLSTM,LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation,Dropout,Add
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import Constant
import numpy as np
from tensorflow.keras.utils import plot_model

HIDDEN_UNITS=200

encoder_inputs = Input(shape=(MAX_LENGTH,))
encoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_art),input_length=MAX_LENGTH,trainable=False,mask_zero=True)(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

decoder_inputs = Input(shape=(MAX_LENGTH,))
decoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix_sum),input_length=MAX_LENGTH,trainable=False,mask_zero=True)(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])

# dense_layer = Dense(VOCAB_SIZE, activation='softmax')
outputs = TimeDistributed(Dense(num_words, activation='softmax'))(decoder_outputs)

model= tf.keras.Model([encoder_inputs, decoder_inputs], outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),    
              metrics=["accuracy"])
print(model.summary())
plot_model(model, to_file='model_seq2seq.png')

Instructions for updating:
Use tf.cast instead.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 68)           0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 68)           0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 68, 300)      3000300     input_7[0][0]                    
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 68, 300)      3000300     input_8[0][0]                    
_____________________________________________________________

In [18]:
num_samples_train = len(train_sentences_Y)
print(num_samples_train)
decoder_output_data_train = np.ones(shape=(num_samples_train, MAX_LENGTH, num_words))

1200


In [19]:
num_samples_test = len(test_sentences_Y)
print(num_samples_test)
decoder_output_data_test = np.ones(shape=(num_samples_test, MAX_LENGTH, num_words))

300


In [25]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

save = ModelCheckpoint('best_model.hdf5', save_best_only=True, monitor='val_acc', mode='max')
history = model.fit([train_sentences_X, train_sentences_Y], 
                     decoder_output_data_train, 
                     epochs=100, 
                     batch_size=32,
                     validation_data=([test_sentences_X, test_sentences_Y], decoder_output_data_test),
                     callbacks=[save])

Train on 1200 samples, validate on 300 samples
Epoch 1/100
1200/1200 [==============================] - 27s 22ms/sample - loss: 0.1422 - acc: 2.5851e-04 - val_loss: 0.1430 - val_acc: 0.0000e+00
Epoch 2/100
1200/1200 [==============================] - 18s 15ms/sample - loss: 0.1422 - acc: 2.5851e-04 - val_loss: 0.1430 - val_acc: 0.0000e+00
Epoch 3/100
1200/1200 [==============================] - 17s 15ms/sample - loss: 0.1422 - acc: 2.5851e-04 - val_loss: 0.1430 - val_acc: 0.0000e+00
Epoch 4/100
1200/1200 [==============================] - 17s 14ms/sample - loss: 0.1422 - acc: 2.5851e-04 - val_loss: 0.1430 - val_acc: 0.0000e+00
Epoch 5/100
1200/1200 [==============================] - 18s 15ms/sample - loss: 0.1422 - acc: 2.5851e-04 - val_loss: 0.1430 - val_acc: 0.0000e+00
Epoch 6/100
 160/1200 [===>..........................] - ETA: 12s - loss: 0.1386 - acc: 0.0000e+00

KeyboardInterrupt: ignored